In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from twilio.rest import Client

def scrape_your_local_ica(url):
    # initialize
    driver = webdriver.Chrome() # use this if you have a macbook
    # use this if you have a PC
    # driver = webdriver.Chrome("C:\Program Files (x86)\chromedriver.exe")
   
    # maximizing window size
    driver.maximize_window()
    driver.get(url) 

    # Click agree on cookies
    agree_button_selector = "button#onetrust-accept-btn-handler"
    WebDriverWait(driver, 3)                                               # wait 3 sec 
    element = driver.find_element(By.CSS_SELECTOR, agree_button_selector) # find the location of the bottom
    element.click()

    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Generate list of soup 
    soup = BeautifulSoup(driver.page_source, "lxml")
    key = "article > div:nth-child(3)"
    containers = soup.select(key)
    offers = [i.text for i in containers]
    
    return offers

def chat_gpt_bot(client, instructions, prompt):
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": prompt}
      ]
    )

    return completion

def send_text(from_number, to_number, text_message):
    message = client_twilio.messages \
        .create(
             from_=from_number,
             body=text_message,
            to=to_number
         )
    
    
# Url to scrape
url = "https://www.ica.se/erbjudanden/ica-supermarket-pelikan-1003447/"

# Setting up a client for ChatGPT & Twilio
client = OpenAI()
client_twilio = Client(os.getenv("account_sid"), os.getenv("auth_token"))

# Describe the role and give instructions to ChatGPT
role = "Du är en kreativ kock som är väldigt duktig på att rekommendera enkla maträtter baserat på tillgängliga ingredienser."
instructions = "Jag vill att du rekommenderar mig maträtter som jag kan laga till middag och sedan göra matlåda av. Du får en lista med veckans rabatterade priser från en matbutik. Du ska använda listan för att skapa 3 enkla recept. Du får också lägga till basvaror (pasta, ris, potatis, lök, couscous, ägg, billigare grönsaker, etc.), även om de inte är med på listan.\nJag vill att du genererar 3 rätter i rubrikform och under varje rubrik/rätt så ska det finnas en lista med ingredienser för 4 portioner. Det är viktigt att enheterna är i enlighet med det metriska systemet. Under listan med ingredienser ska du beskriva väldigt kort hur man tillagar rätten."

# scrape and retrieve offers from ica
context = scrape_your_local_ica(url)
# pass cheap products alongside instructions to chatgpt function to let it do its magic.
completion = chat_gpt_bot(client, role, instructions+"\n"+str(context))

# turn completion into shopping list with chat gpt
shopping_lst = chat_gpt_bot(client, "Hjälpreda", "Generera en inköpslista i punktform med sammanlagda enheter av följande information: "+completion.choices[0].message.content)

# send shopping list to phone using twilio
send_text(os.getenv("my_twilo_number"), os.getenv("my_phone_number"), shopping_lst.choices[0].message.content)

# Write receipes to a txt-file
with open('weekly_recipe.txt', 'w') as file:
    file.write(completion.choices[0].message.content)